In [1]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#from google.colab import files

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Andrés
[nltk_data]    |     Arango\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to C:\Users\Andrés
[nltk_data]    |     Arango\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Andrés
[nltk_data]    |     Arango\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Andrés
[nltk_data]    |     Arango\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_da

In [2]:
#csv = files.upload()



In [3]:
df_amazon = pd.read_csv(r"sentiment labelled sentences\amazon_cells_labelled.txt",sep="\t",header=None,names=['sentence','target'])

In [4]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1000 non-null   object
 1   target    1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
df_imbd = pd.read_csv(r"sentiment labelled sentences\imdb_labelled.txt",sep="\t",header=None,names=['sentence','target'])

In [6]:
df_imbd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  748 non-null    object
 1   target    748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [7]:
df_yelp = pd.read_csv(r"sentiment labelled sentences\yelp_labelled.txt",sep="\t",header=None,names=['sentence','target'])

In [8]:
df_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1000 non-null   object
 1   target    1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [9]:
df_sub1 = df_amazon.append(df_imbd,ignore_index=True)
df= df_sub1.append(df_yelp,ignore_index=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2748 non-null   object
 1   target    2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


In [11]:
df.head()

,sentence,target
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [12]:
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    # Pase el texto a minúscula
    tokens = [token.lower() for token in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    processed_text = ' '.join(tokens)
    return processed_text

In [13]:
df['sentence'] = df['sentence'].apply(preprocess_text)

In [14]:
df.head()

,sentence,target
0,way plug u unless go converter .,0
1,"good case , excellent value .",1
2,great jawbone .,1
3,tied charger conversation lasting 45 minutes.m...,0
4,mic great .,1


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Obtenga su X y su y
X = df['sentence']
y = df['target']

# Codifique su y
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
# Particione el conjunto en entrenamiento y prueba con train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42, train_size=0.2)

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Hágale padding a sus secuencias y obtenga aquellas para entrenamiento y aquellas para prueba

max_words = 10000
max_len = 100
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences_train = tok.texts_to_sequences(X_train)
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_train = pad_sequences(sequences_train,maxlen=max_len)
sequences_matrix_test = pad_sequences(sequences_test,maxlen=max_len)

In [17]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [18]:
dummy_clf = DummyClassifier()
dummy_clf.fit(sequences_matrix_train, y_train)

DummyClassifier()

In [19]:
y_pred = dummy_clf.predict(sequences_matrix_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')


print(f'Exactitud: {accuracy:.2f}')
print(f'Precisión: {precision:.2f}')
print(f'Recuperación: {recall:.2f}')
print(f'Puntuación F1: {f1:.2f}')

Exactitud: 0.50
Precisión: 0.25
Recuperación: 0.50
Puntuación F1: 0.33


In [20]:
from sklearn.model_selection import GridSearchCV

dummy_clf = DummyClassifier()

param_grid = {
    'strategy': ['uniform', 'most_frequent', 'stratified', 'constant'],
    'constant': [0, 1, 2]  
}

grid_search = GridSearchCV(dummy_clf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(sequences_matrix_train, y_train)

best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")



Mejores hiperparámetros: {'constant': 0, 'strategy': 'stratified'}


In [21]:
best_clf = grid_search.best_estimator_
y_pred_best = best_clf.predict(sequences_matrix_test)

accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best, average='weighted')
recall_best = recall_score(y_test, y_pred_best, average='weighted')
f1_best = f1_score(y_test, y_pred_best, average='weighted')

print(f'Exactitud con mejores hiperparámetros: {accuracy_best:.2f}')
print(f'Precisión con mejores hiperparámetros: {precision_best:.2f}')
print(f'Recuperación con mejores hiperparámetros: {recall_best:.2f}')
print(f'Puntuación F1 con mejores hiperparámetros: {f1_best:.2f}')

Exactitud con mejores hiperparámetros: 0.48
Precisión con mejores hiperparámetros: 0.48
Recuperación con mejores hiperparámetros: 0.48
Puntuación F1 con mejores hiperparámetros: 0.48


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [24]:
max_len = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_data = pad_sequences(train_sequences, maxlen=max_len)
test_data = pad_sequences(test_sequences, maxlen=max_len)

In [25]:
def create_rnn_model(units=32):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_len))
    model.add(SimpleRNN(units=units, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

    

In [26]:
rnn_model = create_rnn_model()

In [27]:
rnn_model.fit(train_data, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
7/7 [==============================] - 10s 524ms/step - loss: 0.6912 - accuracy: 0.5308 - val_loss: 0.6938 - val_accuracy: 0.4545
Epoch 2/5
7/7 [==============================] - 2s 273ms/step - loss: 0.6767 - accuracy: 0.6879 - val_loss: 0.6928 - val_accuracy: 0.4727
Epoch 3/5
7/7 [==============================] - 2s 289ms/step - loss: 0.6625 - accuracy: 0.7244 - val_loss: 0.6915 - val_accuracy: 0.4909
Epoch 4/5
7/7 [==============================] - 2s 354ms/step - loss: 0.6438 - accuracy: 0.7927 - val_loss: 0.6896 - val_accuracy: 0.4818
Epoch 5/5
7/7 [==============================] - 2s 293ms/step - loss: 0.6187 - accuracy: 0.8428 - val_loss: 0.6881 - val_accuracy: 0.5182


In [28]:
rnn_model = KerasClassifier(build_fn=create_rnn_model, epochs=5, batch_size=32, verbose=0)


In [29]:
param_grid = {
    'units': [50, 100],
    'epochs': [5, 10],
    'batch_size': [32, 64]
}

grid_search = GridSearchCV(rnn_model, param_grid, cv=3, scoring='accuracy')

# Realizar la búsqueda en los datos de entrenamiento
grid_search.fit(train_data,y_train)

# Imprimir los mejores hiperparámetros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

6/6 [==============================] - 1s 108ms/step
Mejores hiperparámetros: {'batch_size': 32, 'epochs': 10, 'units': 50}


In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_data)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Imprime los resultados con los mejores hiperparámetros
print(f'Exactitud con mejores hiperparámetros: {accuracy:.2f}')
print(f'Precisión con mejores hiperparámetros: {precision:.2f}')
print(f'Recuperación con mejores hiperparámetros: {recall:.2f}')
print(f'Puntuación F1 con mejores hiperparámetros: {f1:.2f}')

69/69 [==============================] - 7s 93ms/step
Exactitud con mejores hiperparámetros: 0.50
Precisión con mejores hiperparámetros: 0.00
Recuperación con mejores hiperparámetros: 0.00
Puntuación F1 con mejores hiperparámetros: 0.00


In [ ]:
def create_lstm_model(units=32):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_len))
    model.add(LSTM(units=units, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

In [ ]:
lstm_model = create_lstm_model()

In [ ]:
lstm_model.fit(train_data, y_train, epochs=5, batch_size=64, validation_split=0.2)

In [ ]:
lstm_model = KerasClassifier(build_fn=create_lstm_model, epochs=5, batch_size=32, verbose=0)

In [ ]:
param_grid = {
    'units': [50, 100],
    'epochs': [5, 10],
    'batch_size': [32, 64]
}

grid_search = GridSearchCV(lstm_model, param_grid, cv=3, scoring='accuracy')

# Realizar la búsqueda en los datos de entrenamiento
grid_search.fit(train_data,y_train)

# Imprimir los mejores hiperparámetros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_data)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Imprime los resultados con los mejores hiperparámetros
print(f'Exactitud con mejores hiperparámetros: {accuracy:.2f}')
print(f'Precisión con mejores hiperparámetros: {precision:.2f}')
print(f'Recuperación con mejores hiperparámetros: {recall:.2f}')
print(f'Puntuación F1 con mejores hiperparámetros: {f1:.2f}')